# Kreditwürdigkeit

## Loading the Data

In [1]:
import pandas as pd 
import numpy as np
from scipy import stats

#Loading the data and naming the columns
def get_data():
    df=pd.read_csv('kredit.dat',sep='\t', header=None, na_values=['?'])
    df.columns=[
    'account','duration','history','purpose','amount','savings','employment','rate','status_and_sex',
    'guarantors','residence','property','age','plans','housing','credits','job','maintenance',
    'telephone','foreigner','creditworthy']
    return df


In [2]:
df=get_data()
df.head()

,account,duration,history,purpose,amount,savings,employment,rate,status_and_sex,guarantors,...,property,age,plans,housing,credits,job,maintenance,telephone,foreigner,creditworthy
0,A14,36,A32,NaN,2299,A63,NaN,4,A93,A101,...,A123,39,A143,A152,1,A173,1,A191,NaN,1
1,A12,18,A32,A46,1239,A65,A73,4,A93,A101,...,A124,61,A143,A153,1,NaN,1,A191,A201,1
2,A13,24,A32,A40,947,A61,A74,4,A93,A101,...,A124,38,A141,A153,1,NaN,2,A191,NaN,2
3,A14,15,A33,A43,1478,A61,A73,4,A94,A101,...,A121,33,A141,A152,2,A173,1,A191,A201,1
4,A14,24,A32,A40,1525,A64,A74,4,A92,A101,...,A123,34,A143,A152,1,A173,2,A192,A201,1


## Understanding the Data

In [3]:
#Prüfen der Daten und Columnnames
import matplotlib.pyplot as plt
from sklearn import preprocessing

def understand_data():

    #Mittelwerte, Standardabweichung, Maxima
    df=get_data()
    np.mean(df)
    np.std(df)
    np.max(df)

    #Ausgabe der einzelnen Werte pro Kategorie für jedes Attribut
    for feature in df.columns:
        print df[feature].value_counts() 
        
    #Anzahl an Attributen und Beispielen
    print df.shape
    
    #Anteil an Kreditwuerdigkeit und Nicht-Kreditwuerdigkeit
    print "Kreditwuerdig"
    print sum(df['creditworthy']==1)
    print "Nicht-Kreditwuerdig"
    print sum(df['creditworthy']==2)
       
    numeric=['duration','amount', 'rate', 'residence', 'age', 'credits', 'maintenance']
    zeroone=['telephone', 'creditworthy']
    missing=['purpose', 'employment', 'job', 'foreigner']
    
    df_nonum=df[df.columns.difference(numeric+zeroone)]
    df_num=df[numeric]

    #Anzahl der fehlenden Werte und Zusammenhang des Attributes mit Zielattribut
    for feature in df[missing]:
        print df[feature].value_counts()  
        print "Missing"
        print sum((df.ix[(df[feature].isnull()==True),'creditworthy'])==1)
    
    #Kategoriale Werte hinsichtlich Kreditwuerdigkeit geordnet
    for feature in df_nonum.columns: 
        plt.figure()
        df_group= df.groupby(['creditworthy'])
        freq=df_group[feature].value_counts()
        (freq.ix[0]/freq.ix[1]).plot(kind='bar')
        plt.title(feature)
        (freq.ix[0]/freq.ix[1]).plot(kind='line', color='red')
        plt.show()
    
    #Verteilungen der nummerischen Werte, getrennt nach Kreditwuerdigkeit    
    for feature in df_num.columns: 
        df.boxplot(feature, by='creditworthy')
        plt.show()
        
    #Notwendigkeit und Effekt der Z-Standardisierung      
    df_num.boxplot()
    plt.show()
    zstand=pd.DataFrame(preprocessing.scale(df_num))
    zstand.columns=numeric
    zstand.boxplot()
    plt.show()
    
    #Range der nummerischen Feature
    for feature in df_num.columns: 
        print feature, ":",  max(df_num[feature]), (df_num[feature])
    

In [4]:
#understand_data()

## Preprocessing of categorical values for linear Models

In [5]:
from sklearn import preprocessing

def get_lin(stand=0):
    
    #Daten abrufen und unterteilen nach nummerisch, fehlend, nicht-nummerisch
    df=get_data()
    X=df.ix[:,df.columns!='creditworthy']
    y=(df.creditworthy ==1)*1  # 1=yes and 0=no

    numeric=['duration','amount', 'rate', 'residence', 'age', 'credits', 'maintenance']
    X_num=X[numeric]
    missing=['purpose', 'employment', 'job', 'foreigner']
    
    #z-Standardisierung
    X_num_scaled =pd.DataFrame(preprocessing.scale(X_num))
    X_num_scaled.columns=numeric
    #print X_num_scaled.head()

    #Erzeugen von Dummyvariablen aus kategorialen vollstaendigen Variablen
    X_nonum=X[X.columns.difference(numeric+missing)]
    X_dum = pd.DataFrame()
    for feature in X_nonum.columns: 
        dummies=pd.get_dummies(X_nonum[feature]) 
        X_dum=pd.concat([X_dum, dummies], axis=1)

    #Frame fuer Lineare Modelle erstellen (wenn stand==1 werden z-stand. Werte genutzt)
    X_lin=pd.concat([X[numeric], X_dum], axis=1)
    if stand==1:
        X_lin=pd.concat([X_num_scaled, X_dum], axis=1)
    X_miss=X[missing]
    
    return X_miss, X_lin, y

In [6]:
X_miss, X_lin, y = get_lin()
print X_miss.columns

Index([u'purpose', u'employment', u'job', u'foreigner'], dtype='object')


## Missing Values (Linear Classification)

In [7]:
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from scipy.stats import mode

def get_full(stand=0, mode=0):
    
    df=get_data()    
    X=pd.DataFrame(df.ix[:,df.columns!='creditworthy']) #Ursprungsframe
    X_miss, X_lin, y=get_lin(stand=stand) #vorverarbeites Frame (dummykodiert und evtl. z-stand.)

    missing=['purpose', 'employment', 'job', 'foreigner']
    X_wasmiss=pd.DataFrame()
    X_filled=pd.DataFrame()
    
    #Alle Attribute mit fehlenden Werten werden nacheinander imputiert
    
    for feature in missing: 
    
        #nur vorhandene Werte zum Training nehmen
        NaNs=(X_miss[feature].isnull()==True)
        NoNaNs=(X_miss[feature].isnull()==False)
        X_nomiss=X_lin[NoNaNs] 
        y_nomiss=X_miss.ix[NoNaNs, feature]
        
        #Labelencoding des Missing-Features         
        le = preprocessing.LabelEncoder() 
        le.fit(np.unique(y_nomiss))
        y_nomiss=le.transform(y_nomiss)

        #Train-Test-Split
        X_train, X_test, y_train, y_test = train_test_split(X_nomiss, y_nomiss, random_state=42)  

        #Logistic Regression (Linear Classification)
        tuned_lm_parameters = [{'C':[0.01, 0.05, 0.1, 0.5, 1.0, 1.1, 1.5, 2.0], 'penalty': ['l1', 'l2']}]      
        lr=GridSearchCV(LogisticRegression(random_state=42),tuned_lm_parameters, 
                        cv=StratifiedKFold(y_train, 5, random_state=42))        
        lr.fit(X_train, y_train) # Trainieren, um Modell zu prüfen

        if mode==0:
            print feature, ':', lr.score(X_test, y_test) # Ausgabe der Accuracy
            lr.fit(X_nomiss, y_nomiss) #Training auf allen Daten
            pred= lr.predict(X_lin[NaNs]) #Missing-Value-Prediction 
            X_miss.ix[NaNs,feature]=list(le.inverse_transform(pred)) #diese Rueckkodieren & einfuegen
        
        #alternative Methode: Modus
        if mode==1:
            correct=0
            modus=le.transform(X[feature].value_counts().argmax())
            for i in range(len(y_test)):
                if (y_test[i]== modus):
                    correct=correct+1
            print feature, ':', (1.*correct)/len(y_test)
            X_miss.ix[NaNs, feature]=X[feature].value_counts().argmax() #diese Rueckkodieren & einfuegen

        #alternative Methode: behalten der fehlenden Werte
        if mode==2:
            print 'es wurden keine fehlenden Werte ersetzt'
            
        #Kodieren, welche vorher NaNs waren
        X_wasmiss=pd.concat([X_wasmiss, NaNs], axis=1)
    
    X_wasmiss.columns=['purpose_nan', 'employment_nan', 'job_nan', 'foreigner_nan']
      
    #Erzeugen eines Dummykodierten-Frames
    numeric=['duration','amount', 'rate', 'residence', 'age', 'credits', 'maintenance']
    X_nonum=df[X.columns.difference(numeric)]
    X_dum = pd.DataFrame()
    for feature in X_nonum.columns: 
        dummies=pd.get_dummies(X_nonum[feature]) 
        X_dum=pd.concat([X_dum, dummies], axis=1)
        
    X_lin=pd.concat([X[numeric], X_dum, X_wasmiss, pd.get_dummies(X_miss)], axis=1)
      
    y=(df.creditworthy ==1)*1  # 1=yes and 0=no
                     
    return X, X_lin, y;
    

In [8]:
print "Fehlende Werte wurden durch lineare Classification ersetzt" 
X, X_lin, y =  get_full(stand=0, mode=0)
print "mit z-Standaridisierung"
X, X_lin, y =  get_full(stand=1, mode=0)
print "Zum Vergleich: Vorhersage durch Modus" 
X, X_lin, y =  get_full(stand=0, mode=1)



Fehlende Werte wurden durch lineare Classification ersetzt
purpose : 0.270531400966
employment : 0.436507936508
job : 0.635416666667
foreigner : 0.94375
mit z-Standaridisierung
purpose : 0.280193236715
employment : 0.47619047619
job : 0.645833333333
foreigner : 0.94375
Zum Vergleich: Vorhersage durch Modus
purpose : 0.246376811594
employment : 0.31746031746
job : 0.625
foreigner : 0.94375


## Loss Function

In [9]:
# Loss-Function, die berücksichtigt, dass fp fünfmal so schlimm wie fn
def weighted_loss(y_test, y_pred):
    y_test = np.array(y_test)
    fp=np.sum(np.logical_and(y_pred!=y_test, y_pred==1))
    fn=np.sum(np.logical_and(y_pred!=y_test, y_pred==0))
    costs=1.*(fp*5 + fn)/len(y_test) #eventuell noch durch sechs teilen
    return costs

## Evaluation mit Kostenmodel (5xFP==FN) und Baseline (immer ja/nein)

In [10]:
def base(y_test):
    print "Baseline: Immer kreditwürdig"
    evaluate(np.ones((len(y_test))), y_test)
    print "Baseline: Immer nicht-kreditwürdig"
    evaluate(np.zeros(len(y_test)), y_test)
    
def evaluate(y_pred, y_test):
    y_test = np.array(y_test)
    
    acc=1.*(np.sum(y_pred==y_test))/len(y_test)
    classif_rate = np.mean(y_pred == y_test) * 100
    
    false=1.*(np.sum(y_pred!=y_test))/len(y_test)
    
    fp=np.sum(np.logical_and(y_pred!=y_test, y_pred==1))
    fn=np.sum(np.logical_and(y_pred!=y_test, y_pred==0))
    
    costs=1.*(fp*5 + fn)/len(y_test) 
    
    print "Accuracy: {}".format(acc) 
    print "False Positives: {}".format(fp) 
    print "False Negatives: {}".format(fn) 
    print "Kosten: {}".format(costs)

## Crossvalidation (nested)

In [23]:
from sklearn.ensemble import RandomForestClassifier

import numpy as np
from sklearn import datasets
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import make_scorer
from sklearn import svm
from sklearn.metrics import precision_recall_curve

def prediction(stand, mode):
    
# Loading the dataset
    print "Alle nummerischen Attribute wurden z-standardisiert."
    print "Fehlende Werte wurden durch lineare Classification ersetzt (Accuracy):" 
    X, X_lin, y =  get_full(stand, mode) 

    #Initialisierung verschiedener Variablen
    y_true = []
    y_pred_lr = []
    y_pred_rf = []
    y_pred_svm = []
    mean_tpr_lr = np.zeros((3, 300))
    mean_fpr_lr = np.zeros((3, 300))
    mean_tpr_rf = np.zeros((3, 300))
    mean_fpr_rf = np.zeros((3, 300))
    mean_tpr_svm = np.zeros((3, 300))
    mean_fpr_svm = np.zeros((3, 300))
    i=0

    #Festlegen der mit Crossvalidierung zu tunenden Parameter
    tuned_rt_parameters = [{'max_depth':[4,5,6,7,8,9,10],
                           'min_samples_leaf':[5,6,7,8,9,10,11,12],                        
                           'criterion':['entropy', 'gini']}]

    tuned_lm_parameters = [{'C':[0.05, 0.5],#, 1.0, 1.5, 2.0], 
                          }]

    score=make_scorer(weighted_loss, greater_is_better=False)
    plt.figure(1)

    #Nested Cross-Validation
    #Stratifizierung -> 30:70 Verhaeltnis bleibt erhalten fuer Test und Train
    for train, test in StratifiedShuffleSplit(y, n_iter=3, test_size=0.3, random_state=42):
        y_true = np.append(y_true, y[test])

        #Random-Forest Classifier mit getunten Hyperparametern
        rf_clf = GridSearchCV(RandomForestClassifier(n_estimators=1000, class_weight={1: 1, 0:5}, 
                                                    random_state=42), 
                           tuned_rt_parameters, cv=StratifiedKFold(y[train], 5), scoring=score)
        rf_clf.fit(X_lin.ix[train], y[train])
        y_pred_rf = np.append(y_pred_rf, rf_clf.predict(X_lin.ix[test]))

        #Logistic Regression Classifier mit getunten Hyperparametern
        clf = GridSearchCV(LogisticRegression(class_weight={1: 1, 0:5}, random_state=42),
                           tuned_lm_parameters, cv=StratifiedKFold(y[train], 5), scoring=score)
        clf.fit(X_lin.ix[train], y[train])
        y_pred_lr = np.append(y_pred_lr, clf.predict(X_lin.ix[test]))

        #Evaluation der Classifier auf den Training-Test-Splits
        print("Beste Parameter fuer Random Forest auf einem der Trainingssets:")
        print(rf_clf.best_estimator_)
        
        print("Werte für den RF auf einem der Trainingssets:")
        evaluate(rf_clf.predict(X_lin.ix[test]), y[test])
        print metrics.classification_report(y[test],rf_clf.predict(X_lin.ix[test]))

        print("Beste Parameter fuer Logistic Regression  auf einem der Trainingssets:")
        print(clf.best_estimator_)

        print("Werte für die LR auf einem der Trainingssets:")
        evaluate(clf.predict(X_lin.ix[test]), y[test])
        print metrics.classification_report(y[test],clf.predict(X_lin.ix[test]))

        #ROC-Kurven
        
        plt.subplot(211)
        #ROC-KURVE fuer Random Forrest
        probs_rf = rf_clf.fit(X_lin.ix[train], y[train]).predict_proba(X_lin.ix[test])
        fpr_rf, tpr_rf, thresholds_rf = roc_curve(y[test], probs_rf[:, 1], drop_intermediate=False)
        roc_auc_rf = auc(fpr_rf, tpr_rf)
        plt.plot(fpr_rf, tpr_rf, lw=1, label='RandFor: ROC (area = %0.2f)' % (roc_auc_rf))

        mean_fpr_rf[i,:]=fpr_rf
        mean_tpr_rf[i,:]=tpr_rf

        #ROC-KURVE fuer logistisch Regression
        probs_lr = clf.fit(X_lin.ix[train], y[train]).predict_proba(X_lin.ix[test])
        fpr_lr, tpr_lr, thresholds_lr = roc_curve(y[test], probs_lr[:, 1], drop_intermediate=False)
        roc_auc_lr = auc(fpr_lr, tpr_lr)
        plt.plot(fpr_lr, tpr_lr, lw=1, label='LogReg: ROC (area = %0.2f)' % (roc_auc_lr))
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc='lower right')
        
        mean_fpr_lr[i,:]=fpr_lr
        mean_tpr_lr[i,:]=tpr_lr

        i=i+1

    #Gemittelte ROC-Kurven
    plt.subplot(212)

    #Gemittelte ROC-KURVE fuer RF
    mean_fpr_rf= np.mean(mean_fpr_rf, axis=0)
    mean_tpr_rf= np.mean(mean_tpr_rf, axis=0)
    mean_auc_rf = auc(mean_fpr_rf, mean_tpr_rf)
    plt.plot(mean_fpr_rf, mean_tpr_rf, 'r',
             label='RF: Mean ROC (area = %0.2f)' % mean_auc_rf, lw=2)

    #Gemittelte ROC-KURVE fuer LR
    mean_fpr_lr= np.mean(mean_fpr_lr, axis=0)
    mean_tpr_lr= np.mean(mean_tpr_lr, axis=0)
    mean_auc_lr = auc(mean_fpr_lr, mean_tpr_lr)
    plt.plot(mean_fpr_lr, mean_tpr_lr, 'b',
             label='LR: Mean ROC (area = %0.2f)' % mean_auc_lr, lw=2)
    
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show()

    #Evaluation der crossvalidierten Ergebnisse 

    print 'Crossvalidierte Ergebnisse fuer Random Forest'
    evaluate(y_pred_rf, y_true)
    print metrics.classification_report(y_true,y_pred_lr)

    print 'Crossvalidierte Ergebnisse fuer Logistic Regression'
    evaluate(y_pred_lr, y_true)
    print metrics.classification_report(y_true,y_pred_lr)

    #Zum Vergleich: Ergebnisse der Baseline
    print 'Ergebnisse der Baseline'
    base(y_true)

In [24]:
print 'Modell mit Standardisierung'
prediction(stand=1, mode=0)


#print 'Modell ohne Standardisierung'
#prediction(stand=0, mode=0)
#print 'Modell ohne Imputation'
#prediction(stand=1, mode=2)

Modell mit Standardisierung
Alle nummerischen Attribute wurden z-standardisiert.
Fehlende Werte wurden durch lineare Classification ersetzt (Accuracy):
purpose : 0.280193236715
employment : 0.47619047619
job : 0.645833333333
foreigner : 0.94375
Beste Parameter fuer Random Forest auf einem der Trainingssets:
RandomForestClassifier(bootstrap=True, class_weight={0: 5, 1: 1},
            criterion='entropy', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
Werte für den RF auf einem der Trainingssets:
Accuracy: 0.563333333333
False Positives: 6
False Negatives: 125
Kosten: 0.516666666667
             precision    recall  f1-score   support

          0       0.40      0.93      0.56        90
          1       0.93      0.40      0.56       210

avg / total       0.77      0.56      0.56 

## Berechung des Finalen Modells

In [25]:
from sklearn.ensemble import RandomForestClassifier

# Loading the dataset
X, X_lin, y = get_full()

#Trainieren des finalen Modells auf allen Daten: 

final_clf = RandomForestClassifier(class_weight={0: 5, 1: 1}, criterion='entropy', max_depth=6, 
                                   min_samples_leaf=7, n_estimators=1000, random_state=42)
         
final_clf.fit(X_lin, y)

purpose : 0.270531400966
employment : 0.436507936508
job : 0.635416666667
foreigner : 0.94375


RandomForestClassifier(bootstrap=True, class_weight={0: 5, 1: 1},
            criterion='entropy', max_depth=6, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)